In [1]:
# PoC for testing in hardware.
import sys
sys.path.insert(1, '../')

# get parameters.
from dev_modules.vcs_model import model_class

2023-08-26 21:50:19.603909: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-26 21:50:19.654949: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-26 21:50:19.655343: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-26 21:50:20.709581: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# IMPORTS

In [2]:
import os
import numpy as np
import pandas as pd
import pickle

import cv2
import matplotlib.pyplot as plt

# Load dataset

In [3]:
with open('../model/Test_files.pkl', 'rb') as file:
    test_files = pickle.load(file)

test_set = pd.DataFrame({"filepaths": test_files})
test_set.filepaths

0      ../dataset/resize_128/test/0_utilizavel/utiliz...
1      ../dataset/resize_128/test/0_utilizavel/utiliz...
2      ../dataset/resize_128/test/0_utilizavel/utiliz...
3      ../dataset/resize_128/test/0_utilizavel/utiliz...
4      ../dataset/resize_128/test/0_utilizavel/utiliz...
                             ...                        
178    ../dataset/resize_128/test/1_defeituoso/defeit...
179    ../dataset/resize_128/test/1_defeituoso/defeit...
180    ../dataset/resize_128/test/1_defeituoso/defeit...
181    ../dataset/resize_128/test/1_defeituoso/defeit...
182    ../dataset/resize_128/test/1_defeituoso/defeit...
Name: filepaths, Length: 183, dtype: object

## Load QT model

In [4]:
qt_model_h = model_class.qt_CNN_MCU()
qt_model = qt_model_h.load_qt_model('../model/qt/models/qt_model.tflite')

## Predictions

In [5]:
y_pred = qt_model_h.qt_predict(test_set)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [6]:
y_pred_df = pd.DataFrame({
    "filename": test_files,
    "prediction": y_pred
})
print(y_pred_df.to_markdown())

|     | filename                                                   |   prediction |
|----:|:-----------------------------------------------------------|-------------:|
|   0 | ../dataset/resize_128/test/0_utilizavel/utiliza (445).jpeg |   0.078125   |
|   1 | ../dataset/resize_128/test/0_utilizavel/utiliza (446).jpeg |   0.269531   |
|   2 | ../dataset/resize_128/test/0_utilizavel/utiliza (447).jpeg |   0.136719   |
|   3 | ../dataset/resize_128/test/0_utilizavel/utiliza (448).jpeg |   0.46875    |
|   4 | ../dataset/resize_128/test/0_utilizavel/utiliza (449).jpeg |   0.00390625 |
|   5 | ../dataset/resize_128/test/0_utilizavel/utiliza (450).jpeg |   0.0078125  |
|   6 | ../dataset/resize_128/test/0_utilizavel/utiliza (451).jpeg |   0.078125   |
|   7 | ../dataset/resize_128/test/0_utilizavel/utiliza (452).jpeg |   0.0390625  |
|   8 | ../dataset/resize_128/test/0_utilizavel/utiliza (453).jpeg |   0.5625     |
|   9 | ../dataset/resize_128/test/0_utilizavel/utiliza (454).jpeg |   0.183

# C IMAGE DATABASE

In [7]:
def get_img_array(path: str) -> tuple:
    name = path.split(os.sep)[-1]
    name = name.replace(' ', '_')
    name = name.replace("(", "")
    name = name.replace(")", "")
    name = name.split('.')[0]
    img_array = "const unsigned char " + name + "[] = "
    return (name, img_array)

In [20]:
arrays_list = list()
names_list = list()

IMPORTS = """// test images as bit arrays.

#include "esp_main.h"
#include "images.h"
#include <freertos/FreeRTOS.h>
"""
with open("./img_arrays.txt", "w") as file:
    file.write(IMPORTS + "\n")

IMG_TO_SAVE = 36
for counter in range(IMG_TO_SAVE):
    idx = int(np.floor(len(test_set.filepaths) / IMG_TO_SAVE)*counter)
    image_path = test_set.filepaths[idx]
    (name, c_array) = get_img_array(image_path)
    names_list.append(name)

    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    array_values = str(img.flatten().tolist()).replace("[", "{").replace("]", "}")
    c_array = c_array + array_values + ";"
    arrays_list.append(c_array)
    
    with open("./img_arrays.txt", "a") as file:
        file.write(c_array + "\n")
    
    print(counter, "-", y_pred[idx], '-', name)

0 - 0.078125 - utiliza_445
1 - 0.0078125 - utiliza_450
2 - 0.109375 - utiliza_455
3 - 0.62109375 - utiliza_460
4 - 0.04296875 - utiliza_465
5 - 0.078125 - utiliza_471
6 - 0.0078125 - utiliza_476
7 - 0.0859375 - utiliza_481
8 - 0.37890625 - utiliza_486
9 - 0.0859375 - utiliza_491
10 - 0.01953125 - utiliza_496
11 - 0.0234375 - utiliza_501
12 - 0.02734375 - utiliza_506
13 - 0.0078125 - utiliza_511
14 - 0.04296875 - utiliza_516
15 - 0.703125 - defeito_673
16 - 0.99609375 - defeito_678
17 - 0.99609375 - defeito_683
18 - 0.99609375 - defeito_688
19 - 0.99609375 - defeito_693
20 - 0.890625 - defeito_698
21 - 0.9921875 - defeito_703
22 - 0.98828125 - defeito_708
23 - 0.99609375 - defeito_713
24 - 0.99609375 - defeito_718
25 - 0.99609375 - defeito_723
26 - 0.9453125 - defeito_728
27 - 0.96875 - defeito_733
28 - 0.99609375 - defeito_738
29 - 0.98046875 - defeito_743
30 - 0.99609375 - defeito_748
31 - 0.99609375 - defeito_753
32 - 0.75390625 - defeito_758
33 - 0.99609375 - defeito_763
34 - 0.9960

In [16]:
ALL_ARRAY = """
const unsigned char* test_img[] = {
"""
with open("./img_arrays.txt", "a") as file:
    file.write(ALL_ARRAY)
    
    for name in names_list:
        file.write(str(name) + ",\n")
    
    file.write("};\n")